In [1]:
import tensorflow as tf
import pickle
import os
import numpy as np
import pandas as pd

rnd_seed = 145
tf.random.set_seed(rnd_seed)

In [2]:


# ------------------------------------------------------------------
# 1. Load the SavedModel (includes the TextVectorization layer)
# ------------------------------------------------------------------
model_save_file = "../../gru_sentimento_6m__GRU_0801_chkp_sent_encoded_200_mzt_220_B32.keras"
model = tf.keras.models.load_model(model_save_file)

# ------------------------------------------------------------------
# 2. (Safety) Re-attach the *exact* vocabulary to the TextVectorization
# ------------------------------------------------------------------
vocab_file = "../../gru_sentimento_6m__GRU_0801_vocb_sent_encoded_200_mzt_220_B32.pkl"
with open(vocab_file, "rb") as f:
    vocab = pickle.load(f)

# Find the TextVectorization layer inside the loaded model
text_vec_layer = [l for l in model.layers if isinstance(l, tf.keras.layers.TextVectorization)][0]
text_vec_layer.set_vocabulary(vocab)

# ------------------------------------------------------------------
# 3. (Optional) Load metadata if you need the constants
# ------------------------------------------------------------------
metadata_file = "../../gru_sentimento_6m__GRU_0801_meta_sent_encoded_200_mzt_220_B32.pkl"
if os.path.exists(metadata_file):
    with open(metadata_file, "rb") as f:
        metadata = pickle.load(f)
    print("Metadata:", metadata)

print("Model ready for prediction!")


Metadata: {'token_count': 30000, 'embed_size': 128, 'num_classes': 3, 'saved_at': '2025-10-29T04:32:35.714609'}
Model ready for prediction!


In [15]:
# Get Data

input_data = pd.read_pickle('../dados/df_ata_sentimento_predict.pkl') 
#input_data = pd.read_json('../dados/df_ata_sentimento.json', orient='records', lines=True)
#input_data['Data'] = pd.to_datetime(input_data['Data']).dt.date
input_data['Texto'] = input_data['Texto'].str.lower()

print(input_data)
print(input_data.iloc[0,1][:100])

          Data                                              Texto  Selic  \
234 2025-05-07  a atualização da conjuntura econômica e do cen...  14.25   
235 2025-06-18  a atualização da conjuntura econômica e do cen...  14.75   
236 2025-07-30  a atualização da conjuntura econômica e do cen...  15.00   
237 2025-09-17  a atualização da conjuntura econômica e do cen...  15.00   

     IPCA  Selic (6m) Data (6m)  
234  5.32         NaN       NaT  
235  5.35         NaN       NaT  
236  5.23         NaN       NaT  
237  5.17         NaN       NaT  
a atualização da conjuntura econômica e do cenário do copom11 o ambiente externo mostrase adverso e 


In [29]:
#0 = dovish
#1 = hawkisk
#2 = neutral
#probs = model.predict(tf.constant([input_data.iloc[0,1]]))
sentiment_map = {0: 'Dovish', 1: 'Hawkisk', 2: 'Neutral'}

probs = model.predict(tf.constant(input_data['Texto'].tolist()))
prediction = tf.argmax(probs, axis=1).numpy()
print(probs)
print(prediction)
input_data['Prediction'] = prediction
input_data['Prediction Text'] = input_data['Prediction'].map(sentiment_map)
print(input_data)




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
[[9.9997252e-01 2.0214731e-05 7.2974567e-06]
 [9.9996394e-01 1.6479769e-05 1.9662908e-05]
 [9.9995834e-01 1.8023871e-05 2.3776018e-05]
 [9.9997705e-01 1.2211467e-05 1.0853659e-05]]
[0 0 0 0]
          Data                                              Texto  Selic  \
234 2025-05-07  a atualização da conjuntura econômica e do cen...  14.25   
235 2025-06-18  a atualização da conjuntura econômica e do cen...  14.75   
236 2025-07-30  a atualização da conjuntura econômica e do cen...  15.00   
237 2025-09-17  a atualização da conjuntura econômica e do cen...  15.00   

     IPCA  Selic (6m) Data (6m)  Prediction Prediction Text  
234  5.32         NaN       NaT           0          Dovish  
235  5.35         NaN       NaT           0          Dovish  
236  5.23         NaN       NaT           0          Dovish  
237  5.17         NaN       NaT           0          Dovish  
